In [18]:
from input_data import *
import csv
from random import random
from sklearn.ensemble import RandomForestClassifier

In [19]:
def get_users():
    with open(INPUT_USERS, 'rb') as users:
        reader = csv.reader(users, delimiter='|')
        return [int(row[0]) for row in reader]

In [37]:
def get_repins(file):
    repins = []
    with open(file, 'rb') as repins_file:
        reader = csv.reader(repins_file, delimiter='|')
        for row in reader:
            repins.append([int(x) for x in row])
    return repins

In [20]:
def get_repins_for_user(user_id, file):
    repins = []
    with open(file, 'rb') as repins_file:
        reader = csv.reader(repins_file, delimiter='|')
        for row in reader:
            if int(row[1]) == user_id:
                repins.append([int(x) for x in row])
    return repins

In [29]:
def get_pins_features_by_repins(repins):
    pins_features = []
    pin_ids = [repin[0] for repin in repins]
    with open(INPUT_PINS, 'rb') as pins:
        reader = csv.reader(pins, delimiter='|')
        for row in reader:
            if int(row[0]) in pin_ids:
                features = [float(x) for x in row[1:1015]]
                pins_features.append(features)
    return pins_features

In [30]:
def get_pins_features_not_in_repins(repins):
    pins_features = []
    pin_ids = [repin[0] for repin in repins]
    with open(INPUT_PINS, 'rb') as pins:
        reader = csv.reader(pins, delimiter='|')
        for row in reader:
            if int(row[0]) not in pin_ids:
                features = [float(x) for x in row[1:1015]]
                pins_features.append(features)

            if len(pins_features) > 500:
                break

    return pins_features

In [31]:
users = get_users()
user_id = users[int(random() * len(users))]
repins = get_repins_for_user(user_id, INPUT_TRAIN_FILE)

x = get_pins_features_by_repins(repins)
y = [1 for i in range(len(x))]

repins = get_repins_for_user(user_id, INPUT_REPINS)
x_n = get_pins_features_not_in_repins(repins)
y_n = [0 for i in range(len(x_n))]

x_all = x + x_n
y_all = y + y_n

print "OK"

OK


In [32]:
clf = RandomForestClassifier(n_estimators=10, n_jobs=4)
clf.fit(x_all, y_all)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [38]:
repins = get_repins(INPUT_TEST_FILE)
if len(repins) == 0:
    print "No repins were found in test set for given user"
    exit(-1)

In [39]:
x_test = get_pins_features_by_repins(repins)
total = len(x_test)

if total == 0:
    print "No pins were loaded based on test set"
    exit(-2)

KeyboardInterrupt: 

In [ ]:
count = 0
for x_hat in x_test:
    y_hat = clf.predict(x_hat)
    
    if y_hat == 1:
        count += 1

print 'Precision =', count * 1. / total